In [1]:
import numpy as np

In [64]:
def genReqs(tot=1000, req_cnt=10):
    return np.random.randint(tot, size=(req_cnt))

def SSTF(v, reqs):
    if len(reqs) == 1:
        return np.hstack([v, reqs])
    
    dist = abs(reqs - v)
    idx_next = np.argmin(dist)
    v_next = reqs[idx_next]
    reqs_next = np.hstack([reqs[:idx_next], reqs[idx_next+1:]])
    
    return np.hstack([v, SSTF(v_next, reqs_next)])

def SCAN(v, reqs, flag=1): #1up 0down
    reqs = np.array(sorted(reqs)).astype(np.int)
    if len(reqs) == 1:
        return np.hstack([v, reqs])
    
    dist = (reqs - v)
    if flag==1:
        dist[dist < 0] = 999999
    else:
        dist[dist > 0] = 999999
    dist = abs(dist)
        
    idx_next = np.argmin(dist)
    v_next = reqs[idx_next]
    reqs_next = np.hstack([reqs[:idx_next], reqs[idx_next+1:]])
    
    if flag==1 and idx_next==len(dist)-1:
        flag = 0
    elif flag==0 and idx_next==0:
        flag = 1
        
    return np.hstack([v, SCAN(v_next, reqs_next, flag)])

reqs, SSTF(reqs[0], reqs[1:]), SCAN(reqs[0], reqs[1:])

(array([856,  68, 286, 194,  43, 238, 581, 150, 477,  58]),
 array([856, 581, 477, 286, 238, 194, 150,  68,  58,  43]),
 array([856,  43,  58,  68, 150, 194, 238, 286, 477, 581]))

In [99]:
import bimpy
import numpy as np

ctx = bimpy.Context()
ctx.init(800, 800, "EXP04")
with ctx:
        bimpy.themes.set_light_theme()
DATA_POINTS = bimpy.Int(30)

## data

##

while not ctx.should_close():
        ctx.new_frame()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 20), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(600, 600), bimpy.Condition.Once)
        bimpy.begin("Disk")
        bimpy.text("Response Time \ Disk Position")
        
        window_pos = bimpy.get_window_pos() + bimpy.Vec2(30, 30)
        
        last_point = None
        for idx, v in enumerate(reqs):
            point = bimpy.Vec2((v/1000*550), idx*10)
            bimpy.add_circle_filled(window_pos + point, 5, 0xAF000000 + 0x4bb43c, 100)
            if idx != 0:
                bimpy.add_line(
                        window_pos + last_point,
                        window_pos + point,
                        0xFFFF0000, 1)
            last_point = point
                
        bimpy.end()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 630), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(600, 110), bimpy.Condition.Once)
        bimpy.begin("Controls")
        
        bimpy.text("Experiment4 Chengxuan Ying 201785071")
        bimpy.input_int("Req Counts", DATA_POINTS)
        if bimpy.button("Generate data"):
            reqs_ = genReqs(tot=1000, req_cnt=DATA_POINTS.value)
            reqs = reqs_
            
        bimpy.same_line()
        if bimpy.button("Orginal"):
            reqs = reqs_
            
        bimpy.same_line()
        if bimpy.button("SSTF"):
            reqs = SSTF(reqs_[0], reqs_[1:])
            
        bimpy.same_line()
        if bimpy.button("SCAN"):
            reqs = SCAN(reqs_[0], reqs_[1:])
            
        t = sum([abs(reqs[idx+1]-reqs[idx]) for idx in range(len(reqs)-1)])
        bimpy.text("Access Cost:" + str(t))
        bimpy.end()

        ctx.render()

KeyboardInterrupt: 